<a href="https://colab.research.google.com/github/Sulaiman29/BDA_HW4/blob/main/Copy_of_BDA2025Spring_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-block alert-info"><b></b>
<h1><center> <font color='black'> Homework 04</font></center></h1>
<h2><center> <font color='black'> Cross-sell / Up-sell using Recommendations</font></center></h2>    
<h2><center> <font color='black'> MTAT.03.319 - Business Data Analytics</font></center></h2>
<h2><center> <font color='black'> University of Tartu - Spring 2024</font></center></h2>
</div>

# Homework instructions

- Please provide the names and student IDs of the team-members (Maximum 2 person) in the field "Team mates" below. If you are not working in a team please insert only your name and student ID.

- Please provide code where ever applicable.

- The accepted submission format is .ipynb file. If you are sharing Colab link, make sure that the privacy settings for the file is public so we can access your code.

- The submission will automatically close on <font color='red'>**04 May at 23:59**</font>, so please make sure to submit before the deadline.

- ONLY one of the teammates should submit the homework and in the submission description the other person's Name and Student ID must be entered. We will grade the homework and the marks and feedback is applied for both the team members. So please communicate with your team member about marks and feedback if you are submit the homework.

- If a question is not clear, please ask us in Moodle ONLY.

- After you have finished solving the Homework, please restart the Kernel and run all the cells to check if there is any persisting issues.

- Plagiarism is <font color='red'>**PROHIBITED**</font>. Any form of plagiarism will be dealt according to the university policy (https://ut.ee/en/content/academic-fraud).

- <font color='red'>**DO NOT CHANGE THE TEMPLATE**</font>

- <font color='red'>**Restart the Kernel and Run all the cells once again after you are done.**</font>
This will ensure that all the cells run without error. You will find an option in the top menu bar under Kernel tab.

**<h2><font color='red'>Team mates:</font></h2>**


<font color='red'>Name Surname:</font>&emsp; MD MURAD ALAHI MISU  <font color='red'>Student ID: </font> C46989


<font color='red'>Name Surname: </font>&emsp; Muhammad Sulaiman Javed  <font color='red'>Student ID: YYYY</font> C47046

### The homework is divided into four sections and the points are distributed as below:
<pre>
- Market Basket Analysis    -> 3 points
- Collaborative Filtering   -> 3.5 points
- Recommendor Systems       -> 1.5 points
- Neural Networks           -> 3 points
__________________________________________
Total                       -> 11 points
</pre>

# 1.  Market Basket Analysis (3 points)

**1.1 Consider the following businesses and think about one case of cross selling and one case of up selling techniques they could use. This question is not restricted to only traditional, standard examples. If you wish you can provide something that you would like these businesses to do. (2 points)**

a. An OnlineTravel Agency like Booking.com or AirBnB

<font color='red'> **Cross selling:**</font> Recommending a Taxi service from the airport to the hotel

<font color='red'> **Up selling:**</font> Upgrading your single room to a Deluxe Room with free breakfast


b. A software company which produces products related to cyber security like Norton, Kaspersky, Avast and similar ones.

<font color='red'> **Cross selling:**</font> Recommending a Password Manager Vault

<font color='red'> **Up selling:**</font> Suggesting upgrading your virus shields to multiple device protection

c. A company that sells cell phones

<font color='red'> **Cross selling:**</font> Recommend accessories combo wireless earbuds + smart watch

<font color='red'> **Up selling:**</font> Suggest a phone model with a higher storage and battery capacity

d. A supermarket like Konsum, Rimi, Maxima etc.

<font color='red'> **Cross selling:**</font> Suggesting charcoal, grills, sauces when customer buys grill meat

<font color='red'> **Up selling:**</font> Offer loyalty/client card discounts on daily gourmet products to attract customers

**1.2.0 One of the techniques which we discussed in the theory lecture for recommendater systems is Market Basket Analysis. The aim is to study the products bought frequently together and to recommend product in bundles. Let's suppose that our client is a retail company that has an online shop. They have given to us the OnlineRetail.csv dataset (we have previously used this dataset in our practice sessions 03). It contains data about the online sales of several products. The client or  wants to know which product bundles to promote. Let us find the 5 association rules with the highest lift.**

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('OnlineRetailPurchase.csv', header=0 )

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


**1.2 Use describe function from pandas to get statistical information about the values in the dataframe. Do you notice something which might not be correct? If so please perform the necessary operations. (Hint: Remember what we did in the practice session 03)(0.25 points)**

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
df.describe()
# Observation: minimum values for both Quantity and UnitPrice are negative or zero, which isn’t valid for actual sales
# cleaning the data
df_clean = df[
    (df['Quantity'] > 0) &
    (df['UnitPrice'] > 0) &
    (~df['InvoiceNo'].astype(str).str.startswith('C'))
]

print("\nAfter cleaning:")
print(df_clean.describe())


After cleaning:
       Quantity  UnitPrice  CustomerID
count   9854.00    9854.00     7608.00
mean       9.15       3.84    15621.15
std       45.76      14.40     1755.24
min        1.00       0.10    12395.00
25%        1.00       1.25    14388.00
50%        3.00       2.51    15640.00
75%        8.00       4.21    17228.00
max     2880.00     887.52    18239.00


**1.3 Create a dataframe name as "Basket", where each row has an distintive value of InvoiceNo and each column has a distinctive Description. The cells in the table contain the count of each item (Description) mentioned in one invoice. For example basket.loc['536365','WHITE HANGING HEART T-LIGHT HOLDER'] has a value of 1 because the product with WHITE HANGING HEART T-LIGHT HOLDER was entered  only once in the invoice 536365. Hint: Remember the function you used in Homework 1 for a similar task or in practice session no.07 (0.25 points)**

In [ ]:
#TODO
basket = pd.crosstab(df_clean['InvoiceNo'], df_clean['Description'])
basket.rename_axis(None, inplace=True)
basket.info()

<class 'pandas.core.frame.DataFrame'>
Index: 424 entries, 536365 to 537224
Columns: 1969 entries,  4 PURPLE FLOCK DINNER CANDLES to ZINC WILLIE WINKIE  CANDLE STICK
dtypes: int64(1969)
memory usage: 6.4+ MB


**1.4 Some products are mentioned more than once in one invoice. You can check the maximum number for each column to verify. Modify your dataframe such that every cell which has a value higher than one will be replaced with 1. If the cell has the value 0 it will remain the same. (0.25 points)**

In [ ]:
''' check and verify those products that occur more than once
max_per_item = basket.max()

# inspect the top 10 products that ever sold more than once on a single invoice
#print(max_per_item.sort_values(ascending=False).head(10))

# see only those that actually exceed 1
multi_buy = max_per_item[max_per_item > 1]
print(multi_buy.sort_values(ascending=False))'''

basket_sets = basket.map(lambda x: 1 if x > 1 else x)

**1.5 We do not need to spend time on calculating the association rules by ourselves as there already exists a package for python to do so, called mlxtend. We are going to use the mlxtend package to find frequent items bought together and then create some rules on what to recomend to a user based on what he/she/they have bought. We have given you the first part of the code which calculates the frequent items bought together.**

In [ ]:
#!pip install mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import mlxtend as ml
import math

In [ ]:
# Mlxtend has implemented Apriori, a popular algorithm for extracting frequent itemsets
# We can change the value of minimum support but it will
# But as well we get less useful results for the next step.
# Setting use_colnames=True to convert the returned integer indices into item names
frequent_itemsets = apriori(basket_sets, min_support=0.03, use_colnames=True)
frequent_itemsets

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.04,( SET 2 TEA TOWELS I LOVE LONDON )
1,0.03,(10 COLOUR SPACEBOY PEN)
2,0.04,(12 PENCILS SMALL TUBE RED RETROSPOT)
3,0.03,(12 PENCILS SMALL TUBE SKULL)
4,0.03,(3 PIECE SPACEBOY COOKIE CUTTER SET)
...,...,...
13602,0.03,"(VINTAGE BILLBOARD LOVE/HATE MUG, WHITE METAL ..."
13603,0.03,"(VINTAGE BILLBOARD LOVE/HATE MUG, WHITE METAL ..."
13604,0.03,"(VINTAGE BILLBOARD LOVE/HATE MUG, WHITE METAL ..."
13605,0.03,"(VINTAGE BILLBOARD LOVE/HATE MUG, WHITE METAL ..."


**Please read the documentation of the associaton rules function in mlextend [here](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/) and then complete the code so we get the 5 rules with the highest lift. Print those rules.  In the output antecedents represent if .... clause and consequents represent else... clause. For example if user bought product basket A then  the algorithm recommends product basket B. (0.25 points)**

In [ ]:
#TODO

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
top5 = rules.sort_values('lift', ascending=False).head(5)

# Print
for _, row in top5.iterrows():
    antecedents = ', '.join(row['antecedents'])
    consequents = ', '.join(row['consequents'])
    print(f"If a user bought {{{antecedents}}}, "
          f"then recommend {{{consequents}}} "
          f"(support={row['support']:.2f}, "
          f"confidence={row['confidence']:.2f}, "
          f"lift={row['lift']:.2f})")

If a user bought {VINTAGE BILLBOARD LOVE/HATE MUG, WHITE HANGING HEART T-LIGHT HOLDER, GLASS STAR FROSTED T-LIGHT HOLDER, CREAM CUPID HEARTS COAT HANGER, VINTAGE BILLBOARD DRINK ME MUG, WOODEN FRAME ANTIQUE WHITE , WOOD 2 DRAWER CABINET WHITE FINISH}, then recommend {RETRO COFFEE MUGS ASSORTED, WOOD S/3 CABINET ANT WHITE FINISH} (support=0.03, confidence=1.00, lift=32.62)
If a user bought {VINTAGE BILLBOARD DRINK ME MUG, WOODEN PICTURE FRAME WHITE FINISH, WOOD 2 DRAWER CABINET WHITE FINISH}, then recommend {VINTAGE BILLBOARD LOVE/HATE MUG, WHITE HANGING HEART T-LIGHT HOLDER, KNITTED UNION FLAG HOT WATER BOTTLE, WOOD S/3 CABINET ANT WHITE FINISH, GLASS STAR FROSTED T-LIGHT HOLDER, RETRO COFFEE MUGS ASSORTED, CREAM CUPID HEARTS COAT HANGER, WOODEN FRAME ANTIQUE WHITE , WHITE METAL LANTERN, RED WOOLLY HOTTIE WHITE HEART.} (support=0.03, confidence=1.00, lift=32.62)
If a user bought {CREAM CUPID HEARTS COAT HANGER, WHITE METAL LANTERN, WOOD 2 DRAWER CABINET WHITE FINISH}, then recommend {V

# 2. Collaborative filtering (3.5 points )

We are going to use Books.csv dataset which contains  ratings from Amazon website and the data has the following features:

UserID: The ID of the users who read the books

BookTitle: The title of the book

Book-Rating: A rating given to the book in a scale from 0 to 10

Below we are going to perform the same steps we did with movies dataset in the practice session

**2.0 Load the dataset and take a look at the books titles. Please pick one of them which you like (or think that you would like) the most.(0.1 points)**

In [ ]:
df_books = pd.read_csv('Books.csv', header=0)
print(df_books['BookTitle'].unique())

# mybook = 'The Da Vinci Code'

['Flesh Tones: A Novel' 'Rites of Passage' 'The Notebook' 'Help!: Level 1'
 'The Amsterdam Connection : Level 4 (Cambridge English Readers)'
 'Les Particules Elementaires' 'A Painted House' 'Lightning'
 'Manhattan Hunt Club' 'Dark Paradise' 'Night Sins' 'At the Edge'
 'Make Them Cry' 'Little Altars Everywhere'
 'How Stella Got Her Groove Back' 'The Law of Love' 'Waiting to Exhale'
 'Birdsong: A Novel of Love and War' 'How to Deal With Difficult People'
 'Change Your Job, Change Your Life: High Impact Strategies for Finding Great Jobs in the Decade Ahead (Change Your Job Change Your Life, 7th ed)'
 'The Golden Rule of Schmoozing'
 'Apricots on the Nile: A Memoir with Recipes'
 'Reise nach Ixtlan. Die Lehre des Don Juan.'
 'The Girl Who Loved Tom Gordon : A Novel' 'The Pillars of the Earth'
 'Southampton Row (Charlotte &amp' 'CLOUT' 'See Jane Run' 'The Dark Half'
 'House of the Sun (Shadowrun)' 'God Game' 'Vater Himmel, Mutter Erde.'
 'Besiegt vom Sturm der Leidenschaft.' 'Sturm der Lieb

**2.1 Our next step will be to perform user based collaborative filtering using KNN algorithm. As KNN algorithm does not accept strings, use a Label Encoder for BookTitle column.After that reshape the books matrix using pivot so every column will be a UserID and every row a BookTitle. (0.45 points)**

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_books['BookTitleEnc'] = le.fit_transform(df_books['BookTitle'])

books_matrix = df_books.pivot_table(
    index='BookTitleEnc',
    columns='UserID',
    values='Book-Rating',
    fill_value=0
)

print(books_matrix.shape)
books_matrix.head()



(336, 6292)


UserID,0,1,2,3,4,5,6,7,8,9,...,6282,6283,6284,6285,6286,6287,6288,6289,6290,6291
BookTitleEnc,,,,,,,,,,,,,,,,,,,,,
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


**2.2 Build a sparse matrix for books data and show it. (0.45 points)**

In [ ]:
from scipy.sparse import csr_matrix

sparse_matrix = csr_matrix(books_matrix.values)

print("Sparse matrix representation:")
print(sparse_matrix)
print(f"\nShape: {sparse_matrix.shape}")
print(f"Non-zero entries: {sparse_matrix.nnz}")

Sparse matrix representation:
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4304 stored elements and shape (336, 6292)>
  Coords	Values
  (1, 6228)	5.0
  (2, 365)	7.0
  (2, 5110)	9.0
  (2, 5226)	10.0
  (2, 6209)	10.0
  (3, 91)	7.0
  (3, 135)	8.0
  (3, 139)	10.0
  (3, 258)	9.0
  (3, 429)	6.0
  (3, 760)	5.0
  (3, 821)	6.0
  (3, 869)	9.0
  (3, 984)	10.0
  (3, 1328)	5.0
  (3, 1425)	10.0
  (3, 1781)	2.0
  (3, 1815)	9.0
  (3, 1820)	9.0
  (3, 1951)	8.0
  (3, 2360)	8.0
  (3, 2384)	8.0
  (3, 2449)	9.0
  (3, 2468)	7.0
  (3, 2956)	5.0
  :	:
  (333, 3411)	4.0
  (333, 3416)	1.0
  (333, 3433)	8.0
  (333, 3445)	5.0
  (333, 3450)	2.0
  (333, 3456)	4.0
  (333, 3461)	8.0
  (333, 3491)	2.0
  (333, 3497)	1.0
  (333, 3501)	4.0
  (333, 3518)	8.0
  (333, 3519)	3.0
  (333, 3527)	7.0
  (333, 3543)	7.0
  (333, 3546)	9.0
  (333, 3547)	7.0
  (333, 3573)	5.0
  (333, 3575)	1.0
  (333, 3581)	7.0
  (333, 3597)	3.0
  (333, 3602)	6.0
  (333, 3666)	3.0
  (333, 3668)	9.0
  (333, 6250)	2.0
  (333, 6277)	9.

**2.3 Build and train two different KNN models (use cosine metric for similarity for both). Recommend top 5 books based on your favourite one from 2.0 (1 points)**

In [ ]:
from sklearn.neighbors import NearestNeighbors

# Model 1
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn.fit(sparse_matrix)

# Model 2
model_knn_2 = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=20, n_jobs=-1)
model_knn_2.fit(sparse_matrix)


mybook = 'The Da Vinci Code'
mybook_index = le.transform([mybook])[0]

distances, indices = model_knn.kneighbors(books_matrix.iloc[mybook_index, :].values.reshape(1, -1), n_neighbors=6)
distances_2, indices_2 = model_knn_2.kneighbors(books_matrix.iloc[mybook_index, :].values.reshape(1, -1), n_neighbors=6)

print("Recommendations using Model 1:")
for i in range(1, len(distances.flatten())):
    print(f"{le.inverse_transform([books_matrix.index[indices.flatten()[i]]])[0]} (distance: {distances.flatten()[i]:.2f})")

print("\nRecommendations using Model 2:")
for i in range(1, len(distances_2.flatten())):
    print(f"{le.inverse_transform([books_matrix.index[indices_2.flatten()[i]]])[0]} (distance: {distances_2.flatten()[i]:.2f})")


Recommendations using Model 1:
The Lovely Bones: A Novel (distance: 0.91)
Harry Potter and the Order of the Phoenix (Book 5) (distance: 0.94)
Southampton Row (Charlotte &amp (distance: 0.94)
A Kiss of Shadows (Meredith Gentry Novels (Paperback)) (distance: 0.95)
Speaking in Tongues (distance: 0.95)

Recommendations using Model 2:
The Lovely Bones: A Novel (distance: 0.91)
Harry Potter and the Order of the Phoenix (Book 5) (distance: 0.94)
Southampton Row (Charlotte &amp (distance: 0.94)
A Kiss of Shadows (Meredith Gentry Novels (Paperback)) (distance: 0.95)
Speaking in Tongues (distance: 0.95)


**2.4 Discuss the results you received. Would you like to read some of the recommended books? Are the recommendation system good? (There is no right or wrong answer in this question) (0.25 points)**

<font color='red'> **Answer:**</font>
The Lovely Bones is definitely on my radar—it’s a suspenseful, character-driven novel, so the thematic mood matches fairly well with a thriller-mystery like The Da Vinci Code.

Both “brute” and “auto” solvers agree, which suggests the nearest-neighbor structure is stable.

The Harry Potter pick is more of a genre outlier, but it often shows up in collaborative filters simply because it’s massively popular and co-rated by many of the same readers.

**2.5 Add a new user (with user “UserID” = 6293) in your data. By training a new KNN model suggest which books should this user read if his ratings are:**

French Cuisine for All: 4


Harry Potter and the Sorcerer's Stone Movie Poster Book: 5


El Perfume: Historia De UN Asesino/Perfume : The Story of a Murderer: 1

**(1. 25 points)**



In [ ]:
new_user_data = {
    'UserID': [6293]*3,
    'BookTitle': [
        'French Cuisine for All',
        "Harry Potter and the Sorcerer's Stone Movie Poster Book",
        'El Perfume: Historia De UN Asesino/Perfume : The Story of a Murderer'
    ],
    'Book-Rating': [4, 5, 1]
}
new_user_df = pd.DataFrame(new_user_data)
df_all = pd.concat([df_books, new_user_df], ignore_index=True)

# Label-encode titles & pivot
le = LabelEncoder()
df_all['BookEnc'] = le.fit_transform(df_all['BookTitle'])

books_matrix = df_all.pivot_table(
    index='BookEnc',
    columns='UserID',
    values='Book-Rating',
    fill_value=0
)

# Build sparse and train two models
sparse_mat = csr_matrix(books_matrix.values)

# Item–item
item_model = NearestNeighbors(metric='cosine',
                             algorithm='brute',
                             n_neighbors=6,
                             n_jobs=-1)
item_model.fit(sparse_mat)

# User–user
user_model = NearestNeighbors(metric='cosine',
                             algorithm='brute',
                             n_neighbors=6,
                             n_jobs=-1)
user_model.fit(sparse_mat.T)

# Extract the new user’s rating vector
new_user_id = 6293
new_user_ratings = books_matrix[new_user_id].values.reshape(1, -1)

# Find the 6 nearest *users* (first neighbor is the user itself)
dist_u, idx_u = user_model.kneighbors(new_user_ratings, n_neighbors=6)

# aggregate the other 5 users’ ratings
sim_users = books_matrix.columns[idx_u.flatten()[1:]]
# average their ratings for each book
avg_scores = books_matrix[sim_users].mean(axis=1)

# Exclude books the user has already rated
already_rated = books_matrix[new_user_id] > 0
avg_scores[already_rated] = -1

# Pick the top 5 book indices
top5_indices = avg_scores.sort_values(ascending=False).head(5).index

# Decode back to titles and print
print("Recommendations for UserID 6293:")
for book_enc in top5_indices:
    print(" •", le.inverse_transform([book_enc])[0])

Recommendations for UserID 6293:
 • Calvin and Hobbes
 • Attack Of The Deranged Mutant Killer Snow Goons
 • The Best of Bombeck: At Wit's End, Just Wait Until You Have Children of Your Own, I Lost Everything in the Post-Natal Depression
 • The Complete Idiot's Guide to the Microsoft Network
 • The Complete Idiots Guide to Getting the Job You Want (W/2 Discs)


# 3. Recommender systems evaluation (1.5 points)

We are going to compare different methods of recommender systems by their RMSE score. One useful package that has several recommender algorithms for Python is [Surprise](https://surprise.readthedocs.io/en/stable/getting_started.html). Below we have split the books dataset into training and test and used the KNNBasic algorithm to predict the ratings for the test set using surprise.

In [ ]:
#!pip install scikit-surprise

In [ ]:
import numpy as np
#from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise import Dataset
from surprise import SVD
from surprise import NormalPredictor
from surprise import KNNBasic

# The reader is necessary for surprise to interpret the ratings
reader = Reader(rating_scale=(0, 10))

# This function loads data from a pandas dataframe into surprise dataset structure
# The columns should always be ordered like this
data = Dataset.load_from_df(df[['UserID', 'BookTitle', 'Book-Rating']], reader)

# Split in trainset and testset
# No need to define the label y because for surprise the last column is always the rating
trainset, testset = train_test_split(data, test_size=.25, random_state=0 )

knn = KNNBasic()
knn.fit(trainset)
predictions = knn.test(testset)
#print('KNN RMSE', accuracy.rmse(predictions))


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

**3.1 After taking a look at surprise documentation and the code above, follow the same steps as with KNN, and predict the ratings in test set using the NormalPredictor which predicts a random rating based on the distribution of the training set. Do the same for SVD which  is a matrix factorization technique. For both of them report RMSE. (We already have imported the functions for you). Which model performed best in regards to RMSE? ** **(1.5 points)**

In [ ]:
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise import Dataset
from surprise import SVD
from surprise import NormalPredictor
from surprise import KNNBasic
# NormalPredictor
np = NormalPredictor()
np.fit(trainset)
predictions_np = np.test(testset)
print('NormalPredictor RMSE', accuracy.rmse(predictions_np))

# SVD
svd = SVD()
svd.fit(trainset)
predictions_svd = svd.test(testset)
print('SVD RMSE', accuracy.rmse(predictions_svd))


<font color='red'> **Answer:**</font>

# 4. Neural Networks (3 Points)

**4.1 We are now going to build a recommender system using Neural Networks. Being this dataset is really small in terms of features you might not see great improvements but it is a good starting point to learn. Please build  one of the neural network architechtures as we did in practice session part 3. You can for example choose the one which had the following layers:**
- 2 Embedding
- 2 Reshape
- 1 Concatenation
- 1 Dense

**Use the Neural Network you built to learn from the train data of part 3 of this homework.  The column UserID should be used as input to your NN for the user embedding layer. For the books embedding layer we will use BookTitle column. Lastly, the ratings will be your target variable. Regarding the evaluation metric for the training phase use RMSE. To make your training fast you can use a batch size of 200 or above. (1.75 points)**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

from keras.layers import Input, Embedding, Flatten, Dot, Dense,multiply, concatenate, Dropout
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam


**4.2 Plot the RMSE values during the training phase, as well as the model loss. Report the best RMSE. Is it better than the RMSE from the models we built in Section 2 and 3 ? (0.75 points)**

<font color='red'> **Answer:**</font>

**4.3 Use your trained model to recommend books for user with ID 6293. (0.5 points)**

## How long did it take you to solve the homework?

* Please answer as precisely as you can. It does not affect your points or grade in any way. It is okay, if it took 0.5 hours or 24 hours. The collected information will be used to improve future homeworks.

<font color='red'> **Answer:**</font>


## What is the level of difficulty for this homework?
you can put only number between $0:10$ ($0:$ easy, $10:$ difficult)

<font color='red'> **Answer:**</font>

## Any other comments regarding the homework?

<font color='red'> **Answer:**</font>